In [1]:
!pip install --upgrade transformers
!pip install datasets
!pip install faiss-gpu
!pip install sentence-transformers
!pip install scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [3]:
from transformers import BartTokenizer, BartForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli',device=0) # Remove device=0


# def classify_document(query, document_content):
#     # Prepare input for the model (NLI task: premise = document, hypothesis = query)
#     # NLI models expect a dictionary input
#     inputs = tokenizer(query, document_content, return_tensors='pt', truncation=True)

#     # Pass the dictionary to the model
#     outputs = model(**inputs)
#     logits = outputs.logits
#     softmax_scores = torch.nn.functional.softmax(logits, dim=-1)
#     entailment_score = softmax_scores[0][2].item()  # 2 is the entailment class
#     return entailment_score
import torch

# Ensure the model is loaded onto the right device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def classify_document(query, document_content):
    # Tokenize with truncation and handle long sequences
    inputs = tokenizer(query, document_content, return_tensors='pt', truncation=True, max_length=512)

    # Move the inputs to the correct device
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Pass the inputs to the model
    with torch.no_grad():  # Disable gradients for faster inference
        outputs = model(**inputs)

    # Extract the logits and calculate softmax scores
    logits = outputs.logits
    softmax_scores = torch.nn.functional.softmax(logits, dim=-1)

    # Get the entailment score (class 2 corresponds to entailment)
    entailment_score = softmax_scores[0][2].item()

    return entailment_score


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset
import os

# Get the current working directory
current_directory = os.getcwd()

# Define the path to the corpus.json file relative to the current directory
file_path = os.path.join(current_directory, "corpus.json")

# Load the dataset using the relative path
dataset = load_dataset("json", data_files=file_path)

def extract_metadata(doc):
    return {
        "title": doc["title"],
        "author": doc.get("author", "Unknown"),
        "url": doc.get("url", "N/A"),
        "source": doc.get("source", "N/A"),
        "category": doc.get("category", "General"),
        "published_at": doc.get("published_at", "N/A"),
        "body": doc["body"]
    }

# Assuming your dataset is named 'train' after loading
corpus = [extract_metadata(doc) for doc in dataset['train']]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained sentence-transformers model
# Rename the model to 'sentence_transformer_model' for clarity and to avoid conflict
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

# Function to encode the corpus
def encode_corpus(corpus):
    document_texts = [doc['body'] for doc in corpus]
    # Use 'sentence_transformer_model' for encoding
    embeddings = sentence_transformer_model.encode(document_texts, convert_to_tensor=False)
    return np.array(embeddings), document_texts

# Step 1: Encode the corpus
corpus_embeddings, document_texts = encode_corpus(corpus)

# Step 2: Build a FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # Using L2 distance (can switch to cosine if needed)

# Add embeddings to the FAISS index
index.add(corpus_embeddings)


# Function to retrieve documents using FAISS
def retrieve_documents_faiss(query, top_k=4):
    # Step 1: Encode the query into an embedding
    # Use 'sentence_transformer_model' for encoding
    query_embedding = sentence_transformer_model.encode([query], convert_to_tensor=False)

    # Step 2: Search FAISS index for top-k similar documents
    distances, indices = index.search(np.array(query_embedding), top_k)

    # Step 3: Retrieve the top-k documents based on the indices
    retrieved_docs = [corpus[i] for i in indices[0]]
    return retrieved_docs

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline

# Load a summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=0)

# def generate_answer(query, documents):
#     ranked_docs = []

#     # Rank documents by entailment score
#     for doc in documents:
#         entailment_score = classify_document(query, doc['body'])
#         ranked_docs.append((doc, entailment_score))

#     # Sort by entailment score in descending order (most relevant first)
#     ranked_docs = sorted(ranked_docs, key=lambda x: x[1], reverse=True)

#     # Get the top entailed documents (e.g., top 2-3)
#     top_docs = [doc for doc, score in ranked_docs[:3]]  # Take top 3 relevant docs

#     if not top_docs:
#         return "No strong evidence found."

#     # Summarize or extract key content from the most entailed documents
#     combined_content = " ".join([doc['body'][:500] for doc in top_docs])  # Limit to 500 chars per doc
#     summary = summarizer(combined_content, max_length=150, min_length=50, do_sample=False)[0]['summary_text']

#     return summary
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import spacy

# Load models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=0)
nlp = spacy.load("en_core_web_sm")

# Load a pre-trained model for query classification
query_classifier_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
query_classifier_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

def classify_query(query):
    inputs = query_classifier_tokenizer(query, return_tensors="pt", truncation=True, padding=True)
    outputs = query_classifier_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()

    # Map prediction to query type (you'll need to define these based on your model's training)
    query_types = ["person", "place", "time", "other"]
    return query_types[prediction]

def generate_answer(query, documents):
    # Your existing code to generate a concise answer
    ranked_docs = sorted(documents, key=lambda x: classify_document(query, x['body']), reverse=True)
    top_docs = ranked_docs[:3]

    if not top_docs:
        return "No relevant information found."

    combined_content = " ".join([doc['body'][:500] for doc in top_docs])
    summary = summarizer(combined_content, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

    return summary

def extract_specific_info(summary, query_type):
    doc = nlp(summary)

    if query_type == "person":
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                return ent.text
    elif query_type == "place":
        for ent in doc.ents:
            if ent.label_ in ["GPE", "LOC"]:
                return ent.text
    elif query_type == "time":
        for ent in doc.ents:
            if ent.label_ == "DATE" or ent.label_ == "TIME":
                return ent.text

    # If no specific entity is found, return the most relevant noun phrase
    for chunk in doc.noun_chunks:
        return chunk.text

    # If no noun phrase is found, return the first sentence
    return summary.split('.')[0]



def format_output(query, answer, documents):
    evidence_list = [{
        "title": doc['title'],
        "author": doc['author'],
        "url": doc['url'],
        "source": doc['source'],
        "category": doc['category'],
        "published_at": doc['published_at'],
        "fact": doc['body'][:200]  # Short excerpt of the content
    } for doc in documents[:3]]  # Only include top 3 documents

    return {
        "query": query,
        "answer": answer,
        "question_type": "inference_query",
        "evidence_list": evidence_list
    }


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# def rag_pipeline(query):
#     # Retrieve the top 4 most relevant documents using FAISS
#     documents = retrieve_documents_faiss(query, top_k=4)

#     # Generate an answer based on the retrieved documents
#     answer = generate_answer(query, documents)

#     # Format the output for consistency
#     return format_output(query, answer, documents)
def rag_pipeline(query):
    # Retrieve relevant documents (assuming this function exists)
    documents = retrieve_documents_faiss(query, top_k=4)

    # Generate a concise answer
    concise_answer = generate_answer(query, documents)

    # Classify the query type
    query_type = classify_query(query)

    # Extract specific information based on the query type
    specific_answer = extract_specific_info(concise_answer, query_type)

    # Format the output
    result = {
        "query": query,
        "answer": f'"{specific_answer}"',
        "question_type": "inference_query",
        "evidence_list": [{
            "title": doc['title'],
            "author": doc['author'],
            "url": doc['url'],
            "source": doc['source'],
            "category": doc['category'],
            "published_at": doc['published_at'],
            "fact": doc['body'][:200]
        } for doc in documents[:3]]
    }

    return result

# Example usage
query = "Who is the individual associated with OpenAI, recognized for both his vision of AI agents and his generosity, and has made headlines in both Fortune and TechCrunch for his controversial departure from the company?"
result = rag_pipeline(query)
print(result)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'query': 'Who is the individual associated with OpenAI, recognized for both his vision of AI agents and his generosity, and has made headlines in both Fortune and TechCrunch for his controversial departure from the company?', 'answer': '"OpenAI"', 'question_type': 'inference_query', 'evidence_list': [{'title': "OpenAI's ex-chairman accuses board of going rogue in firing Altman: 'Sam and I are shocked and saddened by what the board did'", 'author': "Matt O'Brien, The Associated Press", 'url': 'https://fortune.com/2023/11/18/how-did-openai-fire-sam-altman-greg-brockman-rogue-board/', 'source': 'Fortune', 'category': 'business', 'published_at': Timestamp('2023-11-18 15:33:09+0000', tz='UTC'), 'fact': 'ChatGPT-maker Open AI said Friday it has pushed out its co-founder and CEO Sam Altman after a review found he was “not consistently candid in his communications” with the board of directors.\n\n“The boa'}, {'title': 'A timeline of Sam Altman’s firing from OpenAI — and the fallout', 'author'

In [8]:
!pip install fastapi uvicorn
!pip install fastapi uvicorn nest_asyncio pyngrok
!npm install -g localtunnel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [14]:
# Install the required packages if you haven't already
!pip install pyngrok fastapi uvicorn

from pyngrok import ngrok

# Set the Ngrok authentication token
ngrok.set_auth_token("2nRVDpxIs3LFGaE31ly162m361q_kaUK8J65QvzLJEN4gBDc")

# Open a tunnel to the FastAPI app running on port 8000
public_url = ngrok.connect(8000)

print(f"FastAPI is publicly available at: {public_url}")


FastAPI is publicly available at: NgrokTunnel: "https://e327-104-196-125-72.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

# Apply nest_asyncio to allow running uvicorn inside the notebook
nest_asyncio.apply()

# Define the FastAPI app
app = FastAPI()

# Define a Pydantic model for the input query
class QueryRequest(BaseModel):
    query: str

@app.post("/rag_pipeline")
async def rag_pipeline_endpoint(request: QueryRequest):
    query = request.query
    final_answer = rag_pipeline(query)  # Assuming rag_pipeline function is defined elsewhere
    return {"final_answer": final_answer}

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [238]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' trun

INFO:     14.139.34.151:0 - "POST /rag_pipeline HTTP/1.1" 200 OK


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


INFO:     14.139.34.101:0 - "POST /rag_pipeline HTTP/1.1" 200 OK


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


INFO:     14.139.34.151:0 - "POST /rag_pipeline HTTP/1.1" 200 OK


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


INFO:     14.139.34.151:0 - "POST /rag_pipeline HTTP/1.1" 200 OK
